In [ ]:
# default_exp xml

# xml

> Functions to parse XML files

In [ ]:
# export
import pathlib
# import xml.etree.ElementTree
import lxml.etree as ET

Directory where the data (*XML* files) are stored

In [ ]:
directory = pathlib.Path.cwd() / 'samples'
assert directory.exists()
directory

PosixPath('/home/manu/dlsproc/samples')

A (sample) file in that directory

In [ ]:
xml_file = directory / 'sample.xml'
assert xml_file.exists()
xml_file

PosixPath('/home/manu/dlsproc/samples/sample.xml')

## Convenience functions

A function to extract the namespaces *declared* in an *XML* file

In [ ]:
# export
def get_namespaces(input_file: str | pathlib.Path, root_name: str = 'base') -> dict:
    
    tree = ET.parse(input_file)
    
    namespaces = tree.getroot().nsmap
    
    if None in namespaces:
        
        namespaces[root_name] = namespaces.pop(None)
        
    return namespaces

In [ ]:
get_namespaces(xml_file)

{'cbc-place-ext': 'urn:dgpe:names:draft:codice-place-ext:schema:xsd:CommonBasicComponents-2',
 'cac-place-ext': 'urn:dgpe:names:draft:codice-place-ext:schema:xsd:CommonAggregateComponents-2',
 'cbc': 'urn:dgpe:names:draft:codice:schema:xsd:CommonBasicComponents-2',
 'cac': 'urn:dgpe:names:draft:codice:schema:xsd:CommonAggregateComponents-2',
 'ns1': 'urn:oasis:names:specification:ubl:schema:xsd:CommonExtensionComponents-2',
 'base': 'http://www.w3.org/2005/Atom'}

In [ ]:
# hide
import nbdev.export
nbdev.export.notebook2script('10_xml.ipynb')

Converted 10_xml.ipynb.
